In [10]:
import pandas as pd

dataset = pd.read_csv("C:/ZIPcoding/dataset0217ver.csv", low_memory=False)

In [11]:
dataset.isna().sum()

인덱스                      0
접수연도                     0
자치구코드                    0
자치구명                     1
법정동코드                    0
법정동명                     0
지번구분                 33272
지번구분명                33272
본번                   33260
부번                   33260
건물명                  33275
계약일                      0
물건금액(만원)                 0
건물면적(㎡)                  0
토지면적(㎡)              72403
층                    33221
권리구분                545331
취소일                 521050
건축년도                  3123
건물용도                     0
신고구분                295691
신고한 개업공인중개사 시군구명    341435
건물면적정수                   0
이전 실거래가(만원)         131441
자치구최근6개월거래량             47
자치구최근6개월평균금액            47
동최근6개월거래량               47
동최근6개월평균금액              47
dtype: int64

### 자치구명 빈 데이터 채우기

In [13]:
dataset[dataset['자치구명'].isna()]

,인덱스,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,...,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,건물면적정수,이전 실거래가(만원),자치구최근6개월거래량,자치구최근6개월평균금액,동최근6개월거래량,동최근6개월평균금액
101250,101250,2024,26230,NaN,10100,미아동,1.0,대지,,,...,0.0,아파트,중개거래,부산 부산진구,72,NaN,1.0,42450.0,1.0,42450.0


In [14]:
## 법정동명이 미아동이므로 자치구명을 '강북구'로 채우고 자치구 코드는 11305로 바꾸기

# `자치구명`이 NaN이면 '강북구'로 변경
dataset.loc[dataset['자치구명'].isna(), '자치구명'] = '강북구'

# `자치구코드`가 NaN이면 11305로 변경
dataset.loc[dataset['자치구코드'].isna(), '자치구코드'] = 11305

In [15]:
dataset[dataset['자치구명'].isna()]

,인덱스,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,...,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,건물면적정수,이전 실거래가(만원),자치구최근6개월거래량,자치구최근6개월평균금액,동최근6개월거래량,동최근6개월평균금액


### 자치구, 법정동, 본번, 부번 기준으로 그룹화하고 주소 컬럼 추가

In [53]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548407 entries, 0 to 548406
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   인덱스               548407 non-null  int64  
 1   접수연도              548407 non-null  int64  
 2   자치구코드             548407 non-null  int64  
 3   자치구명              548407 non-null  object 
 4   법정동코드             548407 non-null  int64  
 5   법정동명              548407 non-null  object 
 6   지번구분              515135 non-null  float64
 7   지번구분명             515135 non-null  object 
 8   본번                515147 non-null  object 
 9   부번                515147 non-null  object 
 10  건물명               515132 non-null  object 
 11  계약일               548407 non-null  int64  
 12  물건금액(만원)          548407 non-null  int64  
 13  건물면적(㎡)           548407 non-null  float64
 14  토지면적(㎡)           476004 non-null  float64
 15  층                 515186 non-null  float64
 16  권리구분              30

In [55]:
import pandas as pd

# 1. 데이터 그룹화 (first() 대신 agg() 사용)
grouped_df = dataset.groupby(['자치구코드', '법정동코드', '본번', '부번']).agg(lambda x: x.iloc[0]).reset_index()

# 2. `본번`, `부번` 숫자로 변환 (미세한 오차 방지)
grouped_df = grouped_df[pd.to_numeric(grouped_df['본번'], errors='coerce').notna()]
grouped_df = grouped_df[pd.to_numeric(grouped_df['부번'], errors='coerce').notna()]
grouped_df['본번'] = grouped_df['본번'].astype(float).round(0).astype(int)
grouped_df['부번'] = grouped_df['부번'].astype(float).round(0).astype(int)

# 3. 중복 제거 (groupby 후에도 완전히 제거되지 않은 중복 처리)
grouped_df = grouped_df.drop_duplicates(subset=['자치구코드', '법정동코드', '본번', '부번'])

# 4. 새로운 주소 컬럼 추가
grouped_df['주소'] = '서울특별시 ' + grouped_df['자치구명'] + ' ' + grouped_df['법정동명'] + ' ' + grouped_df['본번'].astype(str)
grouped_df.loc[grouped_df['부번'] != 0, '주소'] += '-' + grouped_df['부번'].astype(str)

grouped_df


,자치구코드,법정동코드,본번,부번,인덱스,접수연도,자치구명,법정동명,지번구분,지번구분명,...,건물용도,신고구분,신고한 개업공인중개사 시군구명,건물면적정수,이전 실거래가(만원),자치구최근6개월거래량,자치구최근6개월평균금액,동최근6개월거래량,동최근6개월평균금액,주소
0,11110,10100,1,0,243380,2021,종로구,청운동,1.0,대지,...,연립다세대,중개거래,서울 종로구,82,60000.0,1723.0,63270.5334,281.0,65556.4733,서울특별시 종로구 청운동 1
2,11110,10100,3,0,94108,2024,종로구,청운동,1.0,대지,...,연립다세대,중개거래,서울 종로구,233,NaN,683.0,70080.3616,9.0,147544.4444,서울특별시 종로구 청운동 3
3,11110,10100,3,150,378302,2021,종로구,청운동,1.0,대지,...,연립다세대,NaN,NaN,204,NaN,6085.0,42985.6549,602.0,35488.8405,서울특별시 종로구 청운동 3-150
4,11110,10100,4,3,337061,2021,종로구,청운동,1.0,대지,...,연립다세대,NaN,NaN,216,NaN,5925.0,44228.0022,2744.0,32453.7190,서울특별시 종로구 청운동 4-3
5,11110,10100,4,35,410365,2020,종로구,청운동,1.0,대지,...,연립다세대,NaN,NaN,144,NaN,6536.0,40894.4621,1090.0,41884.8486,서울특별시 종로구 청운동 4-35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113250,11740,11000,699,0,260156,2021,강동구,강일동,1.0,대지,...,아파트,NaN,NaN,84,116000.0,2666.0,75576.2273,163.0,115727.5890,서울특별시 강동구 강일동 699
113252,11740,11000,701,0,272938,2021,강동구,강일동,1.0,대지,...,아파트,NaN,NaN,84,114800.0,2082.0,59167.0072,157.0,105162.3885,서울특별시 강동구 강일동 701
113254,11740,11000,707,0,252572,2021,강동구,강일동,1.0,대지,...,아파트,중개거래,서울 강동구,84,120500.0,2712.0,37362.5033,887.0,51829.4870,서울특별시 강동구 강일동 707
113256,11740,11000,717,0,256037,2021,강동구,강일동,1.0,대지,...,아파트,NaN,NaN,84,118000.0,2932.0,55194.3789,1083.0,35086.0748,서울특별시 강동구 강일동 717


In [18]:
# `자치구코드`, `법정동코드`, `본번`, `부번` 기준으로 그룹화하여 첫 번째 값 유지
grouped_df = dataset.groupby(['자치구코드', '법정동코드', '본번', '부번']).first().reset_index()

# `본번`, `부번` 컬럼에서 숫자가 아닌 값 제거
grouped_df = grouped_df[pd.to_numeric(grouped_df['본번'], errors='coerce').notna()]
grouped_df = grouped_df[pd.to_numeric(grouped_df['부번'], errors='coerce').notna()]

# `본번`, `부번`을 float → int 변환
grouped_df['본번'] = grouped_df['본번'].astype(float).astype(int)
grouped_df['부번'] = grouped_df['부번'].astype(float).astype(int)

# 새로운 주소 컬럼 추가 (부번이 0이면 본번까지만 표시)
grouped_df['주소'] = '서울특별시 ' + grouped_df['자치구명'] + ' ' + grouped_df['법정동명'] + ' ' + grouped_df['본번'].astype(str)
grouped_df.loc[grouped_df['부번'] != 0, '주소'] += '-' + grouped_df['부번'].astype(str)

In [57]:
grouped_df

,자치구코드,법정동코드,본번,부번,인덱스,접수연도,자치구명,법정동명,지번구분,지번구분명,...,건물용도,신고구분,신고한 개업공인중개사 시군구명,건물면적정수,이전 실거래가(만원),자치구최근6개월거래량,자치구최근6개월평균금액,동최근6개월거래량,동최근6개월평균금액,주소
0,11110,10100,1,0,243380,2021,종로구,청운동,1.0,대지,...,연립다세대,중개거래,서울 종로구,82,60000.0,1723.0,63270.5334,281.0,65556.4733,서울특별시 종로구 청운동 1
2,11110,10100,3,0,94108,2024,종로구,청운동,1.0,대지,...,연립다세대,중개거래,서울 종로구,233,NaN,683.0,70080.3616,9.0,147544.4444,서울특별시 종로구 청운동 3
3,11110,10100,3,150,378302,2021,종로구,청운동,1.0,대지,...,연립다세대,NaN,NaN,204,NaN,6085.0,42985.6549,602.0,35488.8405,서울특별시 종로구 청운동 3-150
4,11110,10100,4,3,337061,2021,종로구,청운동,1.0,대지,...,연립다세대,NaN,NaN,216,NaN,5925.0,44228.0022,2744.0,32453.7190,서울특별시 종로구 청운동 4-3
5,11110,10100,4,35,410365,2020,종로구,청운동,1.0,대지,...,연립다세대,NaN,NaN,144,NaN,6536.0,40894.4621,1090.0,41884.8486,서울특별시 종로구 청운동 4-35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113250,11740,11000,699,0,260156,2021,강동구,강일동,1.0,대지,...,아파트,NaN,NaN,84,116000.0,2666.0,75576.2273,163.0,115727.5890,서울특별시 강동구 강일동 699
113252,11740,11000,701,0,272938,2021,강동구,강일동,1.0,대지,...,아파트,NaN,NaN,84,114800.0,2082.0,59167.0072,157.0,105162.3885,서울특별시 강동구 강일동 701
113254,11740,11000,707,0,252572,2021,강동구,강일동,1.0,대지,...,아파트,중개거래,서울 강동구,84,120500.0,2712.0,37362.5033,887.0,51829.4870,서울특별시 강동구 강일동 707
113256,11740,11000,717,0,256037,2021,강동구,강일동,1.0,대지,...,아파트,NaN,NaN,84,118000.0,2932.0,55194.3789,1083.0,35086.0748,서울특별시 강동구 강일동 717


### 위도, 경도 컬럼 추가

구글 스프레드시트 확장 프로그램 사용

In [75]:
address = pd.read_csv("C:/ZIPcoding/address.csv", low_memory=False)
address

,자치구코드,법정동코드,본번,부번,주소,Latitude,Longitude
0,11110,10100,1,0,서울특별시 종로구 청운동 1,NaN,NaN
1,11110,10100,3,0,서울특별시 종로구 청운동 3,37.591066,126.967546
2,11110,10100,3,150,서울특별시 종로구 청운동 3-150,37.590601,126.967965
3,11110,10100,4,3,서울특별시 종로구 청운동 4-3,37.590015,126.967734
4,11110,10100,8,15,서울특별시 종로구 청운동 8-15,37.588491,126.967931
...,...,...,...,...,...,...,...
68385,11740,11000,695,0,서울특별시 강동구 강일동 695,37.559969,127.180761
68386,11740,11000,699,0,서울특별시 강동구 강일동 699,37.559850,127.178826
68387,11740,11000,701,0,서울특별시 강동구 강일동 701,37.561524,127.176878
68388,11740,11000,707,0,서울특별시 강동구 강일동 707,37.561180,127.171665


#### -> 위도, 경도 결측치 존재 (사용한 프로그램에서 검색되지 않은 경우)

### 결측치 확인 및 처리

In [79]:
# Latitude 또는 Longitude에 결측치가 있는 행 찾기
missing_rows = address[address["Latitude"].isna() | address["Longitude"].isna()]
missing_rows

,자치구코드,법정동코드,본번,부번,주소,Latitude,Longitude
0,11110,10100,1,0,서울특별시 종로구 청운동 1,NaN,NaN
5,11110,10100,15,7,서울특별시 종로구 청운동 15-7,NaN,NaN
7335,11215,10500,680,73,서울특별시 광진구 자양동 680-73,NaN,NaN


In [89]:
address.iloc[0,5:] = [37.591286, 126.968196]

In [91]:
address.iloc[5,5:] = [37.590206, 126.969223]

In [93]:
address.iloc[7335,5:] = [37.532929, 127.085833]

In [95]:
missing_rows = address[address["Latitude"].isna() | address["Longitude"].isna()]
missing_rows

,자치구코드,법정동코드,본번,부번,주소,Latitude,Longitude


In [97]:
address.to_csv("C:/ZIPcoding/address.csv", index = False)

In [99]:
address.to_excel("C:/ZIPcoding/address.xlsx", index = False)